<a href="https://colab.research.google.com/github/Rossel/DataQuest_Courses/blob/master/028__Conditional_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COURSE 3/6: STORYTELLING THROUGH DATA VISUALIZATION

# MISSION 4: Conditional_plots

In this mission, we will explore how to ....

## 1. Introduction to Seaborn

So far, we've mostly worked with plots that are quick to analyze and make sense of. Line charts, scatter plots, and bar plots allow us to convey a few nuggets of insights to the reader. We've also explored how we can combine those plots in interesting ways to convey deeper insights and continue to extend the storytelling power of data visualization. In this mission, we'll explore how to quickly create multiple plots that are subsetted using one or more conditions.

We'll be working with the seaborn visualization library, which is built on top of matplotlib. [Seaborn](http://seaborn.pydata.org/) has good support for more complex plots, attractive default styles, and integrates well with the pandas library. Here are some examples of some complex plots that can be created using seaborn:

